## 本例是将V0.2版本的分析结果中，去除晚上02:00-07:55（开始时间）的数据

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
def getEstTripsCount(df,n=5): 
    """
    获取预估数据
    """
    trips_count = df["TRIPS_COUNT"]
    estTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if i < n-1:
            estTripsCount.append(0)
        else:
            estTripsCount.append(sum(trips_count[i+1-n:i+1]))
    return estTripsCount

def getRealTripsCount(df,n=5):
    """
    获取真实数据
    """
    trips_count = df["TRIPS_COUNT"]
    realTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if l-i-1 < n:
            realTripsCount.append(0)
        else:
            realTripsCount.append(sum(trips_count[i+1:i+1+n]))
    return realTripsCount

In [3]:
# 判断发车辆大于空车数（修正后），并排除2:00-7:59的数据
def campareNot2_8(trips_count, vehicle_count_dispersion, hour):
    if trips_count>vehicle_count_dispersion and hour not in [2,3,4,5,6,7]:
        return 1
    else:
        return 0

In [4]:
# 在开始之前，在数据集中添加所需字段（空车差量、预估发车量、实际发车量、预估大于空车标记、真实大于空车标记）
def dealBeforeCalc(dataFrame, carStorageTime, disPersion, writer):
    carStorageCount = int(carStorageTime/5)
    
    # 修改FROM_TIME字段数据类型
    dataFrame["FROM_TIME"] = pd.to_datetime(dataFrame["FROM_TIME"])
    
    # 添加HOUR列
    dataFrame["HOUR"] = dataFrame["FROM_TIME"].dt.hour
    
    # 计算空车差量， 添加到数据表中
    dataFrame["VEHICLE_COUNT_dispersion"] = data.VEHICLE_COUNT*(1+disPersion)
    # 计算预估发车量
    estTripsCount = getEstTripsCount(dataFrame, n=carStorageCount)
    # 计算实际发车量
    realTripsCount = getRealTripsCount(dataFrame, n=carStorageCount)
    
    # 将预估发车量和实际发车量合并为一个新的数据集，然后通过FROM_TIME字段连接到原数据集中
    trips_count_dict = {
        "FROM_TIME":dataFrame["FROM_TIME"],
        "estTripsCount":estTripsCount,
        "realTripsCount":realTripsCount
    }
    # 合并数据集
    df = dataFrame.merge(pd.DataFrame(trips_count_dict), on="FROM_TIME")  
   
    
    # 添加标记列，分别标记预估和真实发车辆大于空车差量的记录
    df["est_lg_VEHICLE_COUNT_dispersion"] = np.vectorize(campareNot2_8)(df["estTripsCount"], df["VEHICLE_COUNT_dispersion"], df["HOUR"])
    df["real_large_VEHICLE_COUNT_dispersion"] = np.vectorize(campareNot2_8)(df["realTripsCount"], df["VEHICLE_COUNT_dispersion"], df["HOUR"])
    
    
    df.to_excel(writer, "{}_{}".format(carStorageTime, disPersion))
    
    
    return df


In [5]:
def calcRate(df):
    est_time = set(df[df["est_lg_VEHICLE_COUNT_dispersion"]==1]["FROM_TIME"])
    real_time = set(df[df["real_large_VEHICLE_COUNT_dispersion"]==1]["FROM_TIME"])
    
    accuracy_rate = len(est_time & real_time)/len(est_time)
    false_positive_rate = len(est_time-real_time)/len(est_time)
    return accuracy_rate, false_positive_rate

In [6]:
data = pd.read_csv("from_06_01_5_minute_stat.csv")
writer = pd.ExcelWriter("analytic_result.xlsx")

carStorageTimes = [20,25,30,35,40]
disPersion = [0.05,0.1,0.15]

result = {
    "carStorageTimes":[],
    "disPersion":[],
    "accuracy_rate":[],
    "false_positive_rate":[]
}

for cst in carStorageTimes:
    for dp in disPersion:
        accuracy_rate, false_positive_rate = calcRate(dealBeforeCalc(data, cst, dp, writer=writer))
        result["carStorageTimes"].append(cst)
        result["disPersion"].append(dp)
        result["accuracy_rate"].append(accuracy_rate)
        result["false_positive_rate"].append(false_positive_rate)
        
df_result = pd.DataFrame(result)
df_result.to_excel(writer, "result")
writer.save()
df_result

,carStorageTimes,disPersion,accuracy_rate,false_positive_rate
0,20,0.05,0.466667,0.533333
1,20,0.10,0.435897,0.564103
2,20,0.15,0.369231,0.630769
3,25,0.05,0.513944,0.486056
4,25,0.10,0.502463,0.497537
5,25,0.15,0.487342,0.512658
6,30,0.05,0.604374,0.395626
7,30,0.10,0.588235,0.411765
8,30,0.15,0.557746,0.442254
9,35,0.05,0.722721,0.277279
